In [ ]:
# Preparación: rutas, importaciones y deps
import os
import sys
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

# Asegurar que el paquete local `ECG_Functions` esté en sys.path
proj_root = os.getcwd()  # debería ser la raíz del repo en el notebook
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

# Importar funciones del paquete
from ECG_Functions import (Cargar_Ecg, Removedor_DC, Detectar_picos_R_AIP, Graficar_ecg_detallado, Graficar_regiones_ecg, Matriz_De_Confusion, Metricas)

# Mostrar versiones y backend
%matplotlib inline
print('Numpy', np.__version__)
import matplotlib; print('Matplotlib', matplotlib.__version__)
import scipy; print('Scipy', scipy.__version__)
import wfdb; print('WFDB', wfdb.__version__)

# Si hace falta instalar algo: uncomment y ejecutar (ejecutar en terminal o pip install desde el kernel)
# !pip install -r requirements.txt

: 

In [ ]:
# Listar variables dentro del archivo ecg.mat para decidir cuál usar
mat_path = 'ecg.mat'
if not os.path.exists(mat_path):
    print('Archivo ecg.mat no encontrado en la raíz del proyecto:', os.getcwd())
else:
    vars_in_mat = sio.whosmat(mat_path)
    print('Variables en ecg.mat (name, shape):')
    for v in vars_in_mat:
        print(v)

In [ ]:
# Cargar la señal usando la función Cargar_Ecg; intentar usar 'ecg_lead' y 'qrs_detections' como nombres comunes
valname = 'ecg_lead'
qrs_name = 'qrs_detections'
try:
    ecg_one_lead, picos_reales, cant_muestras = Cargar_Ecg(mat_path, val=valname, lead=None, qrs_detections=qrs_name)
    print('Se cargó ecg.lead con forma', ecg_one_lead.shape)
    if picos_reales is None:
        print('No se encontró variable qrs_detections en el archivo .mat — picos reales no disponibles')
    else:
        print('Picos reales cargados:', picos_reales.shape)
except Exception as e:
    print('Error al cargar el archivo ecg.mat:', str(e))
    # Intentar inferir alternativas si la carga inicial falla
    print('
Intentando cargar la primera variable encontrada en el .mat como señal...')
    mat_struct = sio.loadmat(mat_path)
    keys = [k for k in mat_struct.keys() if not k.startswith('__')]
    if not keys:
        raise RuntimeError('No se encontraron variables útiles en el .mat')
    alt_key = keys[0]
    print('Usando variable:', alt_key)
    ecg_one_lead = np.asarray(mat_struct[alt_key]).flatten()
    picos_reales = mat_struct.get(qrs_name, None)
    picos_reales = np.asarray(picos_reales).flatten() if picos_reales is not None else None
    cant_muestras = len(ecg_one_lead)

In [ ]:
# Mostrar información básica y graficar un segmento de la señal cruda
fs = 1000.0  # frecuencia de muestreo por defecto
print('Número de muestras:', cant_muestras)
t = np.arange(cant_muestras) / fs

# Graficar segmento de 5 segundos si hay suficientes muestras
seg_seconds = 5
seg_samples = int(seg_seconds * fs)
end_idx = min(cant_muestras, seg_samples)

plt.figure(figsize=(12, 3))
plt.plot(t[:end_idx], ecg_one_lead[:end_idx], color='k')
plt.title('ECG crudo - primeros {:d} segundos'.format(seg_seconds))
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.grid(True)
plt.show()

In [ ]:
# Aplicar Removedor_DC para suavizar la señal y graficar la comparación
ecg_filtered = Removedor_DC(ecg_one_lead, D=64, N=20, window_length=101, polyorder=9)
plt.figure(figsize=(12, 3))
plt.plot(t[:end_idx], ecg_one_lead[:end_idx], label='ECG crudo', alpha=0.6)
plt.plot(t[:end_idx], ecg_filtered[:end_idx], label='ECG filtrado (Golay)', color='red')
plt.title('Comparación ECG crudo vs filtrado - primeros {:d} segundos'.format(seg_seconds))
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Detectar picos R con Detectar_picos_R_AIP y graficar detecciones
peaks_R = Detectar_picos_R_AIP(ecg_filtered, fs=fs, percentile=30, trgt_width=0.09, trgt_min_pattern_separation=0.3)
print('Número de picos detectados:', len(peaks_R))

plt.figure(figsize=(12, 3))
plt.plot(t[:end_idx], ecg_filtered[:end_idx], label='ECG filtrado')
peaks_in_seg = [p for p in peaks_R if p < end_idx]
if peaks_in_seg:
    plt.scatter(np.array(peaks_in_seg)/fs, ecg_filtered[peaks_in_seg], c='r', zorder=3, label='Picos R detectados')
plt.title('Detecciones R - primeros {:d} segundos'.format(seg_seconds))
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Visualización detallada con Poincaré (puede abrir ventanas interactivas si se usa 'matplotlib notebook')
try:
    Graficar_ecg_detallado(ecg_filtered, peaks_R, fs=fs)
except Exception as e:
    print('La función interactiva Graficar_ecg_detallado pudo haber fallado en cabeza del notebook:', e)
    print('Intenta ejecutarla desde un entorno con GUI o cambiar el backend de Matplotlib a notebook interactivo: %matplotlib notebook')

In [ ]:
# Si existen picos reales, calcular la matriz de confusión y métricas
if picos_reales is not None:
    VP, VN, FP, FN, conf = Matriz_De_Confusion(peaks_R, picos_reales, tol=30, cant_muestras=cant_muestras)
    precision, recall, f1, acc = Metricas(conf)
    print('Confusion matrix:')
    print(conf)
    print('
Metrics:')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-score: {f1:.4f}')
    print(f'Accuracy: {acc:.4f}')
else:
    print('No hay picos reales disponibles en el dataset para calcular métricas.')

## Notas de cierre y ajustes
- Si el archivo `.mat` no tiene la variable `ecg_lead`, el notebook intenta cargar la primera variable disponible.
- Ajusta `percentile` y `trgt_min_pattern_separation` en Detectar_picos_R_AIP para mejorar la detección.
- Para interactividad completa con `Graficar_ecg_detallado`, prueba `%matplotlib notebook` o ejecuta el script desde un entorno con GUI.